In [42]:
import numpy as np
import pandas as pd
from ydata_profiling import ProfileReport
pd.set_option('display.max_rows', 2000)

In [97]:
df = pd.read_csv("df_sans_meteo.csv").head(150)

In [4]:
profile = ProfileReport(df, title="Profiling Report")

In [5]:
#profile.to_file("cinema.html")

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
#profile.to_notebook_iframe()

## Observation

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7939 entries, 0 to 7938
Data columns (total 16 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Unnamed: 0             7939 non-null   int64  
 1   title                  7939 non-null   object 
 2   release_date           7939 non-null   object 
 3   genre                  7939 non-null   object 
 4   duration               7939 non-null   object 
 5   director               7939 non-null   object 
 6   producers              7939 non-null   object 
 7   cast                   7939 non-null   object 
 8   nationality            7939 non-null   object 
 9   distributor            7939 non-null   object 
 10  box_office_title       7939 non-null   object 
 11  box_office_first_week  7939 non-null   int64  
 12  press_eval             7939 non-null   object 
 13  viewers_eval           7939 non-null   object 
 14  views                  7939 non-null   float64
 15  budg

In [4]:
# Vérifier s'il y a des valeurs manquantes dans chaque colonne
missing_values = df.isnull().sum()
print(f"Valeurs manquantes :\n{missing_values}")

Valeurs manquantes :
Unnamed: 0               0
title                    0
release_date             0
genre                    0
duration                 0
director                 0
producers                0
cast                     0
nationality              0
distributor              0
box_office_title         0
box_office_first_week    0
press_eval               0
viewers_eval             0
views                    0
budget                   0
dtype: int64


In [9]:
df.describe().transpose()

,count,mean,std,min,25%,50%,75%,max
Unnamed: 0,7939.0,4377.168157,2604.970106,0.0,2117.5,4294.0,6596.0,9113.0
box_office_first_week,7939.0,157466.874795,319289.759308,4.0,6261.0,46320.0,165416.5,3906694.0
views,7939.0,375947.256707,630888.424373,61.0,54085.0,177819.0,449707.0,13541875.0


In [5]:
# Inspecter chaque colonne : 
for value in df:
    print(df[value].value_counts())
    print()

0       1
5820    1
5837    1
5836    1
5834    1
       ..
2830    1
2829    1
2828    1
2827    1
9113    1
Name: Unnamed: 0, Length: 7939, dtype: int64

brothers                 3
la traversée             3
the island               3
home                     3
gold                     2
                        ..
x-men: the last stand    1
planes                   1
driven                   1
kung-fu zohra            1
not in tel-aviv          1
Name: title, Length: 7871, dtype: int64

2013-04-17    15
2013-06-05    15
2016-11-23    15
2017-11-15    15
2013-06-26    14
              ..
2012-05-25     1
2008-09-24     1
2003-07-02     1
2003-06-18     1
2020-02-21     1
Name: release_date, Length: 1245, dtype: int64

Drame                                      1859
Comédie                                     887
Comédie dramatique                          494
Drame, Romance                              198
Thriller                                    193
                               

In [72]:
# Convertir toutes les valeurs de la colonne "genre" en minuscules
df['genre'] = df['genre'].str.lower()

# Supprimer les espaces en début et en fin de chaque genre
df['genre'] = df['genre'].str.strip()

# Supprimer les espaces entre les genres
df['genre'] = df['genre'].str.replace(', ', ',')

In [73]:
df['genre']

0                     action,fantastique,science fiction
1                                                comédie
2                                                  drame
3                                     comédie dramatique
4                                         thriller,drame
5                                       comédie,thriller
6                                       aventure,comédie
7                                         biopic,musical
8                                                comédie
9                                                comédie
10                                 comédie,drame,romance
11                                               comédie
12                                               comédie
13                                                 drame
14                                               comédie
15                                     thriller,policier
16                                                 drame
17                             

In [95]:
# Convertir toutes les valeurs de la colonne "cast" en minuscules et supprimer les espaces en début et en fin
df['cast'] = df['cast'].str.lower().str.strip()

# Supprimer les espaces entre les acteurs
df['cast'] = df['cast'].str.replace(', ', ',')

# Encodage des acteurs
actors_encoding = df['cast'].str.get_dummies(sep=',')
actors_encoding.columns = ['actor_' + col for col in actors_encoding.columns]

# Convertir toutes les valeurs de la colonne "producers" en minuscules et supprimer les espaces en début et en fin
df['producers'] = df['producers'].str.lower().str.strip()

# Supprimer les espaces entre les producteurs
df['producers'] = df['producers'].str.replace(', ', ',')

# Encodage des producteurs
producers_encoding = df['producers'].str.get_dummies(sep=',')
producers_encoding.columns = ['producer_' + col for col in producers_encoding.columns]

# Convertir toutes les valeurs de la colonne "genre" en minuscules et supprimer les espaces en début et en fin
df['genre'] = df['genre'].str.lower().str.strip()

# Supprimer les espaces entre les genres
df['genre'] = df['genre'].str.replace(', ', ',')

# Encodage du genre
genre_encoding = df['genre'].str.get_dummies(sep=',')
genre_encoding.columns = ['genre_' + col for col in genre_encoding.columns]

# Concaténation des DataFrames d'encodage avec le DataFrame principal
df_encoded = pd.concat([df, actors_encoding, producers_encoding, genre_encoding], axis=1)

# Supprimez les colonnes d'origine
df_encoded.drop(columns=['cast', 'producers', 'genre'], inplace=True)

In [100]:
from sklearn.preprocessing import MultiLabelBinarizer

# Convertir les colonnes "actor", "producer" et "genre" en listes de listes
actors_list = [actors.split(',') for actors in df['cast']]
producers_list = [producers.split(',') for producers in df['producer']]
genre_list = [genre.split(',') for genre in df['genre']]

# Créer une instance de MultiLabelBinarizer pour chaque colonne
mlb_actors = MultiLabelBinarizer()
mlb_producers = MultiLabelBinarizer()
mlb_genre = MultiLabelBinarizer()

# Adapter et transformer les étiquettes pour chaque colonne
actors_encoded = mlb_actors.fit_transform(actors_list)
producers_encoded = mlb_producers.fit_transform(producers_list)
genre_encoded = mlb_genre.fit_transform(genre_list)


In [105]:
print(genre_encoded)

[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


In [96]:
genre_encoding.columns

Index(['genre_action', 'genre_animation', 'genre_arts martiaux',
       'genre_aventure', 'genre_biopic', 'genre_comédie',
       'genre_comédie dramatique', 'genre_comédie musicale', 'genre_drame',
       'genre_epouvante-horreur', 'genre_espionnage', 'genre_famille',
       'genre_fantastique', 'genre_guerre', 'genre_historique',
       'genre_judiciaire', 'genre_musical', 'genre_policier', 'genre_romance',
       'genre_science fiction', 'genre_thriller'],
      dtype='object')

In [57]:
df_encoded.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150 entries, 0 to 149
Columns: 683 entries, Unnamed: 0 to genre_Thriller
dtypes: float64(1), int64(672), object(10)
memory usage: 800.5+ KB


In [46]:
import matplotlib as plt

In [106]:
top_distributors = df_encoded['distributor'].value_counts().head(10)

plt.figure(figsize=(10, 6))
plt.bar(top_distributors.index, top_distributors.values)
plt.xlabel('Distributeur')
plt.ylabel('Nombre de films')
plt.title('Top 10 des distributeurs par nombre de films')
plt.xticks(rotation=45)
plt.show()

TypeError: 'module' object is not callable